In [1]:
from utils.data_loader import load_ccpp
X_tr, X_te, y_tr, y_te = load_ccpp(scale=False)
print(X_tr.shape, y_tr[:3].ravel())

'''
(7654, 4)            # размеры X_train
[443.31 490.96 483.94]  # первые 3 значения y_train

7654 — количество объектов, попавших в обучающую выборку.
В исходном наборе 9 568 строк, а load_ccpp() по-умолчанию делит данные
на 80 % / 20 %. 80 % от 9 568 ≈ 7 654 — цифра и совпала.

4 — число признаков (температура T, вакуумное давление V,
абсолютное давление AP, влажность RH).

Cразу после кортежа печатаются первые три истинные значения целевой
переменной PE (выходная мощность турбины в МВт).
'''

(7654, 4) [443.31 490.96 483.94]


'\n(7654, 4)            # размеры X_train\n[443.31 490.96 483.94]  # первые 3 значения y_train\n\n7654 — количество объектов, попавших в обучающую выборку.\nВ исходном наборе 9 568 строк, а load_ccpp() по-умолчанию делит данные\nна 80 % / 20 %. 80 % от 9 568 ≈ 7 654 — цифра и совпала.\n\n4 — число признаков (температура T, вакуумное давление V,\nабсолютное давление AP, влажность RH).\n\nCразу после кортежа печатаются первые три истинные значения целевой\nпеременной PE (выходная мощность турбины в МВт).\n'

In [3]:
import numpy as np
from utils.data_loader import load_ccpp
from utils.metrics import evaluate_optimizer
from core.schedulers import ExponentialDecayScheduler
from core.sgd_variants import Adam

# Определяем loss/grad-функции для MSE-регрессии
def loss_fn(X, y, w):
    ### loss_fn считает MSE (Mean Squared Error) — средний квадрат отклонения
    preds = X @ w
    return float(np.mean((preds - y)**2))

def grad_fn(X, y, w):
    ### даёт аналитический градиент этой MSE по вектору весов w
    preds = X @ w
    return 2 * X.T @ (preds - y) / len(X)

# Загружаем данные
X_tr, X_te, y_tr, y_te = load_ccpp(scale=True)

# Создаём оптимизатор Adam и делаем оценку
opt = Adam(
    learning_rate=1e-3,
    batch_size=256,
    scheduler=ExponentialDecayScheduler(1e-3, lam=1e-4),
    max_iter=200,
    reg={"l2": 1e-4},
)

# Оценкаhttps://archive.ics.uci.edu/
metrics = evaluate_optimizer(
    opt, loss_fn, grad_fn,
    X_tr, y_tr, X_te, y_te,
    eps=1e-6
)
print(metrics)
print("Test RMSE:", metrics["test_rmse"]) #RMSE (Root MSE) — корень из MSE. Это средняя величина ошибки в тех же единицах, что и y.
# Test RMSE ≈ 0.264 говорит, что после стандартизации среднеквадратичное отклонение предсказаний от истины ≈ 0.264σ.
print("Peak memory (MB):", metrics["mem_peak_bytes"] / 1024**2)
print("Number of steps:", metrics["n_steps"])
'''
batch_size=256 → на каждой итерации берётся по 256 случайных точек

max_iter=200 → всего не больше 200 эпох (≈ 200 × ⌈7654/256⌉ ≈ 6000 шагов)

reg={"l2":1e-4} → добавляем L2-регуляризацию

evaluate_optimizer:

    профилирует пиковую память (tracemalloc)

    замеряет время работы

    возвращает

        train/test MSE, RMSE, MAE, R²

        число сделанных шагов (n_steps)

        пиковую память (mem_peak_bytes)

        историю loss по эпохам (history)'''

[[4.]
 [1.]
 [6.]]  
 [[3. ]
 [0.9]
 [4.4]]
Test RMSE: 0.16205388578705696
Peak memory (MB): 0.008691787719726562
Number of steps: 200


'\nbatch_size=256 → на каждой итерации берётся по 256 случайных точек\n\nmax_iter=200 → всего не больше 200 эпох (≈ 200 × ⌈7654/256⌉ ≈ 6000 шагов)\n\nreg={"l2":1e-4} → добавляем L2-регуляризацию\n\nevaluate_optimizer:\n\n    профилирует пиковую память (tracemalloc)\n\n    замеряет время работы\n\n    возвращает\n\n        train/test MSE, RMSE, MAE, R²\n\n        число сделанных шагов (n_steps)\n\n        пиковую память (mem_peak_bytes)\n\n        историю loss по эпохам (history)'